In [49]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [50]:
# Load the training dataset
train_path = "train.csv"
df_train = pd.read_csv(train_path)

In [51]:
import pickle
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LinearRegression

# Load training data
train_path = "train.csv"
df_train = pd.read_csv(train_path)

# Define target variable
target_column = df_train.columns[-1]

# Identify numerical and categorical columns
num_cols = df_train.select_dtypes(include=['number']).columns.tolist()
cat_cols = df_train.select_dtypes(exclude=['number']).columns.tolist()

# Ensure target column is not in numerical columns
if target_column in num_cols:
    num_cols.remove(target_column)

# Drop rows where target is missing
df_train = df_train.dropna(subset=[target_column])

# Convert target column to float
df_train[target_column] = df_train[target_column].astype(float)

# Handle missing values
num_imputer = SimpleImputer(strategy='mean')
df_train[num_cols] = num_imputer.fit_transform(df_train[num_cols])

if cat_cols:
    cat_imputer = SimpleImputer(strategy='most_frequent')
    df_train[cat_cols] = cat_imputer.fit_transform(df_train[cat_cols])

# Encode categorical variables
label_encoders = {}
for col in cat_cols:
    le = LabelEncoder()
    df_train[col] = le.fit_transform(df_train[col].astype(str))
    label_encoders[col] = le

# Splitting features and target
X = df_train.drop(columns=[target_column])
y = df_train[target_column]

# Scale numerical features
scaler = StandardScaler()
X[num_cols] = scaler.fit_transform(X[num_cols])

# Train the Linear Regression Model
model = LinearRegression()
model.fit(X, y)

# Save the trained model
with open("linear.pkl", "wb") as file:
    pickle.dump(model, file)

print("Model has been trained and saved as 'linear.pkl'")


Model has been trained and saved as 'linear.pkl'


In [52]:
# Load the training dataset
test_path = "test.csv"
df_test = pd.read_csv(test_path)

In [53]:

df_test.isnull().sum()

x    0
y    0
dtype: int64

In [54]:
# Load the trained model from the pickle file
with open(model_filename, 'rb') as file:
    loaded_model = pickle.load(file)

# Prepare the test data (assuming 'x' is the feature column)
X_test = df_test[['x']]

# Make predictions on the test data
y_test_pred = loaded_model.predict(X_test)

# Add predictions to the test dataframe
df_test['y_pred'] = y_test_pred

print(df_test.head())

    x          y       y_pred
0  77  79.775152  2279.308013
1  21  23.177279   657.949363
2  22  25.609262   686.902196
3  20  17.857388   628.996530
4  36  41.849864  1092.241858


In [57]:
import pickle
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, LabelEncoder

# Handle missing values in the test dataset
num_imputer = SimpleImputer(strategy='mean')
cat_imputer = SimpleImputer(strategy='most_frequent')

num_cols = df_test.select_dtypes(include=['number']).columns.tolist()
cat_cols = df_test.select_dtypes(exclude=['number']).columns.tolist()

df_test[num_cols] = num_imputer.fit_transform(df_test[num_cols])
if cat_cols:
    df_test[cat_cols] = cat_imputer.fit_transform(df_test[cat_cols])

# Encode categorical variables using LabelEncoder
label_encoders = {}
for col in cat_cols:
    le = LabelEncoder()
    df_test[col] = le.fit_transform(df_test[col].astype(str))
    label_encoders[col] = le

# Scale numerical features using StandardScaler
scaler = StandardScaler()
df_test[num_cols] = scaler.fit_transform(df_test[num_cols])

# Ensure the test dataset only contains the features used during training
df_test = df_test[X.columns]

# Load the trained model
with open("linear.pkl", "rb") as f:
    model = pickle.load(f)

# Make predictions
test_predictions = model.predict(df_test)

# Save predictions
df_test["Predictions"] = test_predictions
output_path = "test_predictions.csv"
df_test.to_csv(output_path, index=False)

print(f"Predictions saved to {output_path}")


Predictions saved to test_predictions.csv


C:\Users\rahim\AppData\Local\Temp\ipykernel_22956\3704481018.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test[num_cols] = num_imputer.fit_transform(df_test[num_cols])
C:\Users\rahim\AppData\Local\Temp\ipykernel_22956\3704481018.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test[num_cols] = scaler.fit_transform(df_test[num_cols])


In [58]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Calculate evaluation metrics
mae = mean_absolute_error(df_test['Predictions'], y_test_pred)
mse = mean_squared_error(df_test['Predictions'], y_test_pred)
rmse = np.sqrt(mse)
r2 = r2_score(df_test['Predictions'], y_test_pred)

print(f"Mean Absolute Error (MAE): {mae}")
print(f"Mean Squared Error (MSE): {mse}")
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"R-squared (R²): {r2}")

Mean Absolute Error (MAE): 1474.7608052980115
Mean Squared Error (MSE): 2806865.5348275886
Root Mean Squared Error (RMSE): 1675.3702679788694
R-squared (R²): -3347.4165221976764


In [ ]:
##The evaluation metrics for the model's performance on the test dataset indicate 
# that the model did not perform well. The Mean Absolute Error (MAE) is 1474.76, 
# which suggests that on average, the model's predictions are off by approximately 
# 1474.76 units from the actual values. The Mean Squared Error (MSE) is 2806865.53, 
# and the Root Mean Squared Error (RMSE) is 1675.37, both of which are quite high,
# indicating significant errors in the predictions. Additionally, the R-squared (R²) 
# value is -3347.42, which is extremely poor and suggests that the model fails to 
# explain the variance in the target variable. An R² value this low indicates that 
# the model's predictions are worse than simply using the mean of the target variable 
# as the prediction. Overall, these metrics highlight that the model's predictions are 
# not reliable and that there is substantial room for improvement.